In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd

In [3]:
from SimpleEmulator.ParseEtxOutputs import toHexV, parseOutputPackets, parseIdle, parseHeaderWords, parseHeaderWord0, parseHeaderWord1, parsePacketHeader

from SimpleEmulator.ParseErxInputs import parseDAQLink, crc

from SimpleEmulator.SimulatorCounters import simulateCounters
from SimpleEmulator.LoadI2C import parseI2C

from SimpleEmulator.I2CColumnInfo import i2c_columns, new_i2c_columns

from SimpleEmulator.ECONDSimpleEmulator import ECOND_Simple_Emulator

In [4]:
#load data from simvision UVM run
df=pd.read_csv('data/simvision.csv',dtype=object)

df.SimTime=df.SimTime.astype(int)
df.set_index('SimTime',inplace=True)

#drop last row (typically a double count when exporting form simvision)
df.drop(df.index[-1],inplace=True)



In [5]:
#parse data into inputs/outputs/FC/i2c
dfInput=df.loc[:,[f'tb_top.test_harness.econd.Aligner_Out_eRX{i}[31:0]' for i in range(12)]]
dfOutput=df.loc[:,[f'tb_top.test_harness.econd.ECOND_P1_Right_u.buffer_top.Ser_Ctrl_u.genblk1[{i}].Word_Aligner_j.in[31:0]' for i in range(6)]]

dfInput.columns=[f'eRx_{i}' for i in range(12)]
dfOutput.columns=[f'eTx_{i}' for i in range(6)]

dfFC=df.loc[:,['tb_top.test_harness.econd.BCR_FCmd',
       'tb_top.test_harness.econd.EBR_FCmd',
       'tb_top.test_harness.econd.ECR_FCmd',
       'tb_top.test_harness.econd.L1A_FCmd',
       'tb_top.test_harness.econd.LINK_RESET_ECON_D_FCmd',
       'tb_top.test_harness.econd.LINK_RESET_ROC_D_FCmd',
       'tb_top.test_harness.econd.NZS_FCmd',
       'tb_top.test_harness.econd.OCR_FCmd',
       'tb_top.test_harness.econd.SPARE_0_FCmd']].astype(int)

dfFC.columns=['BCR','EBR','ECR','L1A','LINK_RESET_ECON_D','LINK_RESET_ROC_D','NZS','OCR','SPARE_0']


i2cValues=df.loc[:,i2c_columns].iloc[0]
i2cValues.index=new_i2c_columns

i2c=parseI2C(i2cValues)


In [6]:
#locate headers and idles
isHeader=np.vectorize(lambda x: x.startswith(i2c.ROC_HdrMarker) &( x.endswith('5') | x.endswith('2')))(dfInput.values).T[i2c.ERx_active].T.sum(axis=1)>i2c.Match_thresh
isIdle=np.vectorize(lambda x: x.startswith(i2c.ROC_FirstSyncHeader+i2c.ROC_SyncBody) | x.startswith(i2c.ROC_SyncHeader+i2c.ROC_SyncBody))(dfInput.shift(fill_value='00000000').values.T[i2c.ERx_active].T).sum(axis=1)>i2c.Match_thresh


startIndices=np.argwhere(isHeader&isIdle).flatten()
dfFC['isHeader']=(isHeader&isIdle).astype(int)

dfCounters=simulateCounters(dfFC)

In [7]:
#cheat, looking at rr values from chip outputs
rr=np.where(np.vectorize(lambda x: x.startswith(i2c.IdlePattern))(dfOutput['eTx_0']),
         (dfOutput['eTx_0'].apply(int,base=16).values>>6)&3,
         np.nan)
rr=pd.DataFrame({"RR":rr}).ffill().astype(int)
RRbits=np.vectorize(lambda x: f'{x:02b}')(rr.values.flatten())

In [8]:
emulatorPackets=ECOND_Simple_Emulator(dfInput, dfCounters, startIndices, i2c, RRbits)

simulatorPackets=parseOutputPackets(dfOutput,i2c)

In [9]:
### First Check, are there the same number of packets in the outputs
if len(emulatorPackets)==len(simulatorPackets):
    print(f'Found {len(emulatorPackets)} packets in both emumulator and UVM')
else:
    print(f'Mismatch in number of packets: {len(emulatorPackets)} packets in emumulator and {len(simulatorPackets)} packets in UVM')
    


Found 86 packets in both emumulator and UVM


In [10]:
#check all the packets, seeing that they are in agreement

def p_f(passFail):
    if passFail:
        return "Pass"
    else:
        return "FAIL"

    
runPass=True
for i in range(len(emulatorPackets)):
    if len(emulatorPackets[i])==len(simulatorPackets[i]):
        fullMatch=(emulatorPackets[i][:]==simulatorPackets[i][:]).all()
        subpacketsMatch=(emulatorPackets[i][2:]==simulatorPackets[i][2:]).all()
        headerMatch=(emulatorPackets[i][:2]==simulatorPackets[i][:2]).all()
        hdrFields_em=parseHeaderWords(emulatorPackets[i])
        hdrFieldsNoRR_em=hdrFields_em[:8]+hdrFields_em[9:-2]
        hdrFields_sim=parseHeaderWords(simulatorPackets[i])
        hdrFieldsNoRR_sim=hdrFields_sim[:8]+hdrFields_sim[9:-2]
        headerMatchNoRR=hdrFieldsNoRR_em==hdrFieldsNoRR_sim
        if not (subpacketsMatch and )
        print(f'{i:03d}:  FullMatchNoRR: {p_f(subpacketsMatch and headerMatchNoRR)}; FullMatch: {p_f(fullMatch)}; SubpacketMatch: {p_f(subpacketsMatch)};  hdrMatchNoRR: {p_f(headerMatchNoRR)}; hdrMatch: {p_f(headerMatch)}')
    else:
        print(f'{i:03d}:  Mismatch in packet lengths; Emulator Packet = {len(emulatorPackets[i])} words; Simulator Packet = {len(simulatorPackets[i])} words; ')


000:  FullMatchNoRR: Pass; FullMatch: Pass; SubpacketMatch: Pass;  hdrMatchNoRR: Pass; hdrMatch: Pass
001:  FullMatchNoRR: Pass; FullMatch: Pass; SubpacketMatch: Pass;  hdrMatchNoRR: Pass; hdrMatch: Pass
002:  FullMatchNoRR: Pass; FullMatch: Pass; SubpacketMatch: Pass;  hdrMatchNoRR: Pass; hdrMatch: Pass
003:  FullMatchNoRR: Pass; FullMatch: Pass; SubpacketMatch: Pass;  hdrMatchNoRR: Pass; hdrMatch: Pass
004:  FullMatchNoRR: Pass; FullMatch: Pass; SubpacketMatch: Pass;  hdrMatchNoRR: Pass; hdrMatch: Pass
005:  FullMatchNoRR: Pass; FullMatch: Pass; SubpacketMatch: Pass;  hdrMatchNoRR: Pass; hdrMatch: Pass
006:  FullMatchNoRR: Pass; FullMatch: Pass; SubpacketMatch: Pass;  hdrMatchNoRR: Pass; hdrMatch: Pass
007:  FullMatchNoRR: Pass; FullMatch: FAIL; SubpacketMatch: Pass;  hdrMatchNoRR: Pass; hdrMatch: FAIL
008:  FullMatchNoRR: Pass; FullMatch: Pass; SubpacketMatch: Pass;  hdrMatchNoRR: Pass; hdrMatch: Pass
009:  FullMatchNoRR: Pass; FullMatch: Pass; SubpacketMatch: Pass;  hdrMatchNoRR: P